In [4]:
import splitfolders as split

In [5]:
# Constants
SPLIT_SEED  = 2048
TRAIN_RATIO = 0.80
TEST_RATIO  = 0.05
VAL_RATIO   = 0.15

In [6]:
# Split the data
split.ratio(input = "../asl_alphabet_train/", output = "../asl_alphabet_data/", seed = SPLIT_SEED, ratio = (TRAIN_RATIO, VAL_RATIO, TEST_RATIO))

Copying files: 87000 files [06:43, 215.85 files/s]
